This Project was completed using OpenCv only, If you hope to find Deep Learning and Machine Learning in this project then you won't find any!!!

This Project was done using 6 steps:

    
1) Find the color range of the target object and save it

2) Apply the correct morphological operation to reduce noise in the video

3) Detect and track the colored object with contour detection.

4) Find the object's x,y location coordinates to draw on the screen

5) Add a Wiper Functionality to wipe off the Whole screen

6) Add an Eraser Functionality to erase parts of the drawing

Before moving on I strongly recommend you to check Edge detection, Countor Detection, BacgroundSubtractor and look how to find them in OpenCV and all the Parameteres they accept

And do check about RBG, HSV

Ok lets get Started!!!

In [1]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
import time

In [2]:

cap =cv2.VideoCapture(0)
pen = cv2.resize(cv2.imread('images/pen.png',1), (50, 50)) #read and resize

eraser = cv2.resize(cv2.imread('images/eraser.jpg',1), (50, 50)) # read and resize



kernel= np.ones((5,5), np.uint8)
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
backgroundobject = cv2.createBackgroundSubtractorMOG2(detectShadows = False) # This helps to toggle between Pen and Eraser
background_threshold = 600
switch = 'Pen'
last_switch = time.time()

write_on=None # Where we write
x1,y1=0,0
noise =500
wiper_thresh =40000
clear=False

while(1):
    _,frame =cap.read()
    frame =cv2.flip(frame,1)
    if write_on is None:
        write_on= np.zeros_like(frame)
    top_left = frame[0: 50, 0: 50] #top left of the screen or say frame

    foregroundmask = backgroundobject.apply(top_left)
    switch_thresh = np.sum(foregroundmask==255)
    if switch_thresh>background_threshold  and (time.time()-last_switch)>1:
        last_switch = time.time()
        if switch =='Pen':
            switch ='Eraser'
        else:
            switch = 'Pen'
            
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    lower_range =np.array([0,157,59]) # You can set your own value by running sethsv.py file
    upper_range=np.array([179,255,255])
        
        
    mask = cv2.inRange(hsv, lower_range, upper_range)
    mask=cv2.erode(mask, kernel, iterations =1)# Please do set your iteration and kernel
    mask= cv2.dilate(mask, kernel,iterations=2)
    
    retval, contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    if contours and cv2.contourArea(max(contours, key= cv2.contourArea))>noise:
        
        c =max(contours, key=cv2.contourArea)
        
        x,y,w,h =cv2.boundingRect(c)
        area= cv2.contourArea(c)
        
        if x1==0 and y1==0:
            x1,y1=x,y
        else:
            if switch=='Pen':
                write_on= cv2.line(write_on,(x1,y1),(x,y),[255,0,0],4)
            else:
                cv2.circle(write_on, (x2,y2),20,(0,0,0),-1)
                
        x1,y1 = x,y
        
        if area>wiper_thresh:
            cv2.putText(write_on,"Clearing Canvas",(0,200),
                        cv2.FONT_HERSHEY_SIMPLEX,2, (0,0,255), 1, cv2.LINE_AA)
            clear =True
    else:
        x1,y1 =0,0
    _, mask = cv2.threshold(cv2.cvtColor(write_on,cv2.COLOR_BGR2GRAY),20,255, cv2.THRESH_BINARY)
    foreground = cv2.bitwise_and(write_on, write_on, mask=mask)
    background = cv2.bitwise_and(frame, frame, mask = cv2.bitwise_not(mask))
    frame = cv2.add(foreground, background) # Now we add both frame and the canvas where we wrote the thing
    
    if switch!='Pen':
        cv2.circle(frame, (x1, y1), 20, (255,255,255), -1)
        frame[0: 50, 0: 50] = eraser #image of eraser
    else:
        frame[0: 50, 0: 50] = pen #image of pen

        
        
    cv2.imshow('image',frame)
    k = cv2.waitKey(1)& 0xFF
    if k==27:
        break
    if clear ==True:
        time.sleep(1)
        write_on=None
        clear =False
    if k == ord('c'):
        write_on = None #after you press c all the written thing will vanish

cv2.destroyAllWindows()
cap.release()